# Importando dados processados

In [1]:
import pickle 
import datasets
import pandas as pd
import os

/home/kenzosaki/miniconda3/envs/bonito/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
datasets.__version__

'4.0.0'

In [3]:
with open("/exp/kenzosaki/data/LeanDL-HPC/my_data.pickle", "rb") as file:
    split_train = pickle.load(file)
    split_eval = pickle.load(file)
    split_test = pickle.load(file)

In [4]:
train_df = split_train.to_pandas()
eval_df = split_eval.to_pandas()
test_df = split_test.to_pandas()

# Exploração dos dados

In [5]:
train_df.columns

Index(['hash_id', 'tema_id', 'tema', 'palavras_chave', 'uf_tema_info',
       'uf_pesquisador', 'nome_programa', 'sigla_entidade_ensino',
       'nome_producao', 'nome_subtipo_producao', 'nome_area_concentracao',
       'nome_linha_pesquisa', 'nome_projeto', 'descricao_palavra_chave',
       'descricao_resumo', 'descricao_abstract', 'descricao_keyword',
       'data_titulacao', 'nome_grau_academico',
       'nome_grande_area_conhecimento', 'nome_area_conhecimento',
       'nome_subarea_conhecimento', 'nome_especialidade', 'modelo_nivel',
       'modelo_explicacao', '__index_level_0__'],
      dtype='object')

In [6]:
train_df['tema'].nunique(), len(train_df)

(437, 33620)

In [7]:
train_df['modelo_nivel'].value_counts()/len(train_df)

modelo_nivel
BAIXA    0.478614
MEDIA    0.308418
ALTA     0.212968
Name: count, dtype: float64

In [8]:
# TODO: remover
# train_df = train_df.sample(5000, random_state=2025) # para teste

# Preparando inputs para o ModernBert

In [9]:
train_df = train_df.dropna()

In [10]:
train_df['palavras_chave'] = train_df['palavras_chave'].apply(lambda array: ", ".join(array.tolist()))
eval_df['palavras_chave'] = eval_df['palavras_chave'].apply(lambda array: ", ".join(array.tolist()))
test_df['palavras_chave'] = test_df['palavras_chave'].apply(lambda array: ", ".join(array.tolist()))

In [11]:
# input_cols = ['nome_producao', 'nome_area_concentracao', 'nome_linha_pesquisa', 'nome_projeto', 'descricao_palavra_chave', 'descricao_resumo', 'tema', 'palavras_chave']
input_cols = ['nome_projeto', 'descricao_palavra_chave', 'descricao_resumo', 'tema', 'palavras_chave']
input_cols_mapping = {col: col.replace('_', ' ').capitalize() for col in input_cols}
input_cols_mapping

{'nome_projeto': 'Nome projeto',
 'descricao_palavra_chave': 'Descricao palavra chave',
 'descricao_resumo': 'Descricao resumo',
 'tema': 'Tema',
 'palavras_chave': 'Palavras chave'}

In [12]:
def create_input_from_row(row: pd.Series) -> str:
  input_str = ""

  for col, formated_name in input_cols_mapping.items():
    input_str += f"{formated_name}: {row[col].capitalize()}\n"

  return input_str

In [13]:
train_df["bert_raw_inputs"] = [create_input_from_row(row) for _, row in train_df.iterrows()]
eval_df["bert_raw_inputs"] = [create_input_from_row(row) for _, row in eval_df.iterrows()]
test_df["bert_raw_inputs"] = [create_input_from_row(row) for _, row in test_df.iterrows()]

# Splits de treino e teste

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [15]:
label_encoder = LabelEncoder()
train_df['modelo_nivel'] = label_encoder.fit_transform(train_df['modelo_nivel'])
eval_df['modelo_nivel'] = label_encoder.transform(eval_df['modelo_nivel'])
test_df['modelo_nivel'] = label_encoder.transform(test_df['modelo_nivel'])


# Hugginface

In [16]:
from datasets import Dataset
from transformers import AutoTokenizer, ModernBertForSequenceClassification
from transformers import DataCollatorWithPadding

In [17]:
train_ds = Dataset.from_dict(
    {
        "text": train_df['bert_raw_inputs'].tolist(),
        "label": train_df['modelo_nivel'].tolist()
    }
)

eval_ds = Dataset.from_dict(
    {
        "text": eval_df['bert_raw_inputs'].tolist(),
        "label": eval_df['modelo_nivel'].tolist()
    }
)

test_ds = Dataset.from_dict(
    {
        "text": test_df['bert_raw_inputs'].tolist(),
        "label": test_df['modelo_nivel'].tolist()
    }
)

In [18]:
MAX_LEN = 1024
FREEZE_ENCODER = False
model = "answerdotai/ModernBERT-base"
tokenizer = AutoTokenizer.from_pretrained(model, truncation_side='left')
model = ModernBertForSequenceClassification.from_pretrained(model, num_labels=3, max_position_embeddings=MAX_LEN) # talvez 2048

Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
model

ModernBertForSequenceClassification(
  (model): ModernBertModel(
    (embeddings): ModernBertEmbeddings(
      (tok_embeddings): Embedding(50368, 768, padding_idx=50283)
      (norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (drop): Dropout(p=0.0, inplace=False)
    )
    (layers): ModuleList(
      (0): ModernBertEncoderLayer(
        (attn_norm): Identity()
        (attn): ModernBertAttention(
          (Wqkv): Linear(in_features=768, out_features=2304, bias=False)
          (rotary_emb): ModernBertRotaryEmbedding()
          (Wo): Linear(in_features=768, out_features=768, bias=False)
          (out_drop): Identity()
        )
        (mlp_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): ModernBertMLP(
          (Wi): Linear(in_features=768, out_features=2304, bias=False)
          (act): GELUActivation()
          (drop): Dropout(p=0.0, inplace=False)
          (Wo): Linear(in_features=1152, out_features=768, bias=False)
        )
      

In [20]:
if FREEZE_ENCODER:
    print(f"- Freezing encoder layers")
    for param in model.model.parameters():
        param.requires_grad = False

# Analisando número de tokens no conjunto de dados inteiro

In [21]:
def get_num_tokens(text: str) -> int:
  return len(tokenizer.encode(text))

In [22]:
sampled_df = train_df.sample(1000, random_state=2025)
# sampled_df = dados_df

In [23]:
sampled_df["num_tokens"] = sampled_df["bert_raw_inputs"].apply(get_num_tokens)

In [24]:
sampled_df["num_tokens"].describe()

count    1000.000000
mean      910.950000
std       270.799701
min       241.000000
25%       731.750000
50%       876.000000
75%      1068.500000
max      4304.000000
Name: num_tokens, dtype: float64

In [25]:
sampled_df["num_tokens"].describe()

count    1000.000000
mean      910.950000
std       270.799701
min       241.000000
25%       731.750000
50%       876.000000
75%      1068.500000
max      4304.000000
Name: num_tokens, dtype: float64

In [26]:
sampled_df["num_tokens"].quantile(0.70)

np.float64(1021.3)

# Pre-processamento de Datasets

In [27]:
from transformers import DataCollatorWithPadding

In [28]:
def tokenizer_function(example):
  # Em caso de uma tarefa de classificação de pares de texto, modificar este valor de retorno
  # truncation=True, padding="max_length", max_length=123 para truncar e padronizar os tamanhos de tokens!!!
  return tokenizer(
      example["text"], truncation=True, max_length=MAX_LEN
  )

In [29]:
# Tokenizando todos os elementos do conjunto de dados em batches
train_ds = train_ds.map(tokenizer_function, batched=True)
eval_ds = eval_ds.map(tokenizer_function, batched=True)
test_ds = test_ds.map(tokenizer_function, batched=True)

Map: 100%|██████████| 4203/4203 [00:00<00:00, 5176.62 examples/s]


In [30]:
# Mantendo apenas colunas cujos nomes coincidam com os métodos forward dos transformers
train_ds = train_ds.remove_columns(column_names=["text"])
train_ds = train_ds.rename_column("label", "labels")
train_ds = train_ds.with_format("torch")

eval_ds = eval_ds.remove_columns(column_names=["text"])
eval_ds = eval_ds.rename_column("label", "labels")
eval_ds = eval_ds.with_format("torch")

test_ds = test_ds.remove_columns(column_names=["text"])
test_ds = test_ds.rename_column("label", "labels")
test_ds = test_ds.with_format("torch")

In [31]:
train_ds

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 33620
})

In [32]:
# Collator para Dynamic Padding
collator = DataCollatorWithPadding(tokenizer, padding="longest")

# Implementando trainer com loss customizada

In [33]:
from transformers import Trainer, TrainingArguments
import torch
import torch.nn as nn
from typing import Any, Optional, Union

In [34]:
train_df['modelo_nivel'].value_counts().loc[[0,1,2]]

modelo_nivel
0     7160
1    16091
2    10369
Name: count, dtype: int64

In [35]:
# inverso da frequência (ajustado para só 3 classes)
class_counts = torch.tensor(train_df['modelo_nivel'].value_counts().loc[[0,1,2]].values, dtype=torch.float, device="cuda" if torch.cuda.is_available() else "cpu")
class_weights = 1.0 / class_counts
class_weights = class_weights / class_weights.sum() * len(class_counts)

In [36]:
# Supondo que você já tenha contado os exemplos por classe
# class_counts = [100, 300, 50]  # Exemplo para 3 classes
# class_weights = [1.0 / c for c in class_counts]  # inversamente proporcional à frequência
# class_weights = torch.tensor(class_weights, dtype=torch.float).to("cuda")

loss_fn = nn.CrossEntropyLoss(weight=class_weights)


class WeightedTrainer(Trainer):

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch: Optional[torch.Tensor] = None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss = loss_fn(logits, labels) # usa a loss_fn definida acima

        return (loss, outputs) if return_outputs else loss

# Configurando Trainer

In [37]:
from transformers import get_cosine_schedule_with_warmup, get_constant_schedule
from torch.optim import AdamW
from transformers import EarlyStoppingCallback
from codecarbon import EmissionsTracker # para calcular emissões de CO2
from transformers import set_seed
import math
import torch
import time

In [38]:
balanced_loss = False
bs = 32
acc_steps = 1
epochs = 10
lr = 2e-4
wd = 1e-3
patience = 2
seed = 42
output_dir = f"/exp/kenzosaki/data/LeanDL-HPC/models/modern_bert_{epochs}epochs_{bs}bs_{lr}lr_{wd}wd_{acc_steps}accsteps/"
train_code_carbon_out = 'modernbert_fine_tuning_emissions_training.csv'
inference_code_carbon_out = 'modernbert_fine_tuning_emissions_inference.csv'

warmup_steps = math.ceil((len(train_ds)/bs) * epochs * 0.1) #10% of train data for warm-up
train_steps = int(epochs * len(train_ds)/bs)
es_callback = EarlyStoppingCallback(early_stopping_patience=patience)

In [39]:
warmup_steps, train_steps

(1051, 10506)

In [40]:
import numpy as np
from sklearn.metrics import classification_report
# from datasets import load_metric

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    metrics_dict = classification_report(labels, predictions, output_dict=True)

    return {
        "accuracy": metrics_dict["accuracy"],
        "precision": metrics_dict["macro avg"]["precision"],
        "recall": metrics_dict["macro avg"]["recall"],
        "f1": metrics_dict["macro avg"]["f1-score"]
    }


In [41]:
# optimizer = AdamW(model.parameters(), lr=lr)
# # scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=train_steps, num_cycles=0.5)
# scheduler = get_constant_schedule(optimizer) # testar este tbm

In [ ]:
set_seed(seed)

In [43]:
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=epochs,                                                                              # total number of training epochs
    per_device_train_batch_size=bs,                                                                       # batch size per device during training
    per_device_eval_batch_size=bs,                                                                        # batch size for evaluation
    # warmup_steps=warmup_steps,                                                                            # number of warmup steps for learning rate scheduler
    eval_strategy="epoch",
    save_strategy="epoch",                                                                                # evaluation interval
    logging_dir=f"{output_dir}/logs",                                                                                 # directory for storing logs
    save_total_limit=patience+1,                                                                              # checkpoint save interval
    report_to='none',
    gradient_accumulation_steps=acc_steps,
    metric_for_best_model='loss',
    greater_is_better=False,
    load_best_model_at_end=True,
    bf16=True,
    learning_rate=lr,
    weight_decay=wd,
    warmup_ratio=0.03,
    logging_steps=100,
    logging_strategy='steps',
    log_level="info",
    seed=seed
)


In [44]:
if balanced_loss:
    print(f"Using balanced loss with class weights: {class_weights}")
    trainer = WeightedTrainer(
        model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=eval_ds,
        compute_metrics=compute_metrics,
        # optimizers=(optimizer, scheduler),
        data_collator=collator
    )
else:
    print(f"Using standard loss")
    trainer = Trainer(
        model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=eval_ds,
        compute_metrics=compute_metrics,
        # optimizers=(optimizer, scheduler),
        data_collator=collator
    )

trainer.add_callback(es_callback)

Using auto half precision backend


Using standard loss


In [45]:
start_time = time.time()
tracker = EmissionsTracker(output_file=train_code_carbon_out)

tracker.start()
trainer.train()
emissions: float = tracker.stop()

print("\n\nTotal de emissões (detalhes em emissions.csv): ",emissions)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nTempo total de execução: {elapsed_time:.2f} segundos")

[codecarbon WARNING @ 15:47:17] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 15:47:17] [setup] RAM Tracking...
[codecarbon INFO @ 15:47:17] [setup] CPU Tracking...
[codecarbon WARNING @ 15:47:18] We saw that you have a Intel(R) Core(TM) i9-14900KF but we don't know it. Please contact us.
[codecarbon WARNING @ 15:47:18] We will use the default power consumption of 4 W per thread for your 32 CPU, so 128W.
[codecarbon WARNING @ 15:47:18] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist at /sys/class/powercap/intel-rapl/subsystem to measure CPU

[codecarbon INFO @ 15:47:18] CPU Model on constant consumption mode: Intel(R) Core(TM) i9-14900KF
[codecarbon WARNING @ 15:47:18] No CPU tracking mode found. Falling back on CPU load mode.
[codecarbon INFO @ 15:47:18] [setup] GPU Tracking...
[codecarbon INFO @ 15:47:18] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 15:47:1

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.983500,0.966582,0.539615,0.377871,0.444164,0.382739
2,0.903700,0.898410,0.563407,0.510935,0.501584,0.470730
3,0.815900,0.875528,0.590768,0.555759,0.538089,0.528013
4,0.729800,0.894588,0.596717,0.582960,0.548062,0.555983
5,0.660900,0.944268,0.561504,0.579126,0.543404,0.555919


[codecarbon INFO @ 15:47:37] Energy consumed for RAM : 0.000164 kWh. RAM Power : 38.0 W
[codecarbon INFO @ 15:47:37] Delta energy consumed for CPU with cpu_load : 0.000055 kWh, power : 12.813782412800002 W
[codecarbon INFO @ 15:47:37] Energy consumed for All CPU : 0.000055 kWh
[codecarbon INFO @ 15:47:37] Energy consumed for all GPUs : 0.000916 kWh. Total GPU Power : 206.11647938678934 W
[codecarbon INFO @ 15:47:37] 0.001135 kWh of electricity used since the beginning.
[codecarbon INFO @ 15:47:52] Energy consumed for RAM : 0.000317 kWh. RAM Power : 38.0 W
[codecarbon INFO @ 15:47:52] Delta energy consumed for CPU with cpu_load : 0.000052 kWh, power : 12.811583950400001 W
[codecarbon INFO @ 15:47:52] Energy consumed for All CPU : 0.000107 kWh
[codecarbon INFO @ 15:47:52] Energy consumed for all GPUs : 0.001867 kWh. Total GPU Power : 228.1136456989476 W
[codecarbon INFO @ 15:47:52] 0.002290 kWh of electricity used since the beginning.
[codecarbon INFO @ 15:48:07] Energy consumed for RAM 



Total de emissões (detalhes em emissions.csv):  0.03893726942621769

Tempo total de execução: 5161.55 segundos


# Avaliação final

In [46]:
from sklearn.metrics import classification_report

In [47]:
# trainer._load_from_checkpoint("/exp/kenzosaki/data/LeanDL-HPC/models/modern_bert_10epochs_32bs_0.0002lr_0.001wd_1accsteps/checkpoint-6306")

In [48]:
tracker = EmissionsTracker(output_file=inference_code_carbon_out)
tracker.start()
preds = trainer.predict(test_ds)
emissions = tracker.stop()
print(f"- Emissões durante a inferência: {emissions}")

[codecarbon WARNING @ 17:13:18] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 17:13:18] [setup] RAM Tracking...
[codecarbon INFO @ 17:13:18] [setup] CPU Tracking...
[codecarbon WARNING @ 17:13:20] We saw that you have a Intel(R) Core(TM) i9-14900KF but we don't know it. Please contact us.
[codecarbon WARNING @ 17:13:20] We will use the default power consumption of 4 W per thread for your 32 CPU, so 128W.
[codecarbon WARNING @ 17:13:20] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist at /sys/class/powercap/intel-rapl/subsystem to measure CPU

[codecarbon INFO @ 17:13:20] CPU Model on constant consumption mode: Intel(R) Core(TM) i9-14900KF
[codecarbon WARNING @ 17:13:20] No CPU tracking mode found. Falling back on CPU load mode.
[codecarbon INFO @ 17:13:20] [setup] GPU Tracking...
[codecarbon INFO @ 17:13:20] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 17:13:2

[codecarbon INFO @ 17:13:38] Energy consumed for RAM : 0.000164 kWh. RAM Power : 38.0 W
[codecarbon INFO @ 17:13:39] Delta energy consumed for CPU with cpu_load : 0.000055 kWh, power : 12.804270755840001 W
[codecarbon INFO @ 17:13:39] Energy consumed for All CPU : 0.000055 kWh
[codecarbon INFO @ 17:13:39] Energy consumed for all GPUs : 0.000981 kWh. Total GPU Power : 220.61855688650192 W
[codecarbon INFO @ 17:13:39] 0.001200 kWh of electricity used since the beginning.
[codecarbon INFO @ 17:13:53] Energy consumed for RAM : 0.000317 kWh. RAM Power : 38.0 W
[codecarbon INFO @ 17:13:54] Delta energy consumed for CPU with cpu_load : 0.000052 kWh, power : 12.8054072144 W
[codecarbon INFO @ 17:13:54] Energy consumed for All CPU : 0.000107 kWh
[codecarbon INFO @ 17:13:54] Energy consumed for all GPUs : 0.001932 kWh. Total GPU Power : 228.30725281838718 W
[codecarbon INFO @ 17:13:54] 0.002355 kWh of electricity used since the beginning.
[codecarbon INFO @ 17:13:58] Energy consumed for RAM : 0.

- Emissões durante a inferência: 0.0002649696439047386


In [49]:
label_encoder.classes_

array(['ALTA', 'BAIXA', 'MEDIA'], dtype=object)

In [50]:
print(classification_report(test_ds['labels'], preds.predictions.argmax(axis=-1), target_names=label_encoder.classes_))

              precision    recall  f1-score   support

        ALTA       0.59      0.57      0.58       863
       BAIXA       0.63      0.83      0.72      2056
       MEDIA       0.39      0.20      0.26      1284

    accuracy                           0.59      4203
   macro avg       0.54      0.53      0.52      4203
weighted avg       0.55      0.59      0.55      4203



In [51]:
print(classification_report(test_ds['labels'], preds.predictions.argmax(axis=-1)))

              precision    recall  f1-score   support

           0       0.59      0.57      0.58       863
           1       0.63      0.83      0.72      2056
           2       0.39      0.20      0.26      1284

    accuracy                           0.59      4203
   macro avg       0.54      0.53      0.52      4203
weighted avg       0.55      0.59      0.55      4203

